## 테스트

In [4]:
import requests

url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=json&iStartUnitCode=101&iEndUnitCode=103"
req = requests.get(url)
print(req)

<Response [200]>


In [5]:
json_df = req.json()
json_df

{'count': 520,
 'pageNo': 1,
 'numOfRows': 10,
 'pageSize': 52,
 'realUnitTrtmVO': [{'pageNo': None,
   'numOfRows': None,
   'startUnitCode': '101 ',
   'endUnitCode': '103 ',
   'stdDate': '20230807',
   'sumTmUnitTypeCode': None,
   'startEndStdTypeCode': '2',
   'timeAvg': '7.8',
   'timeMax': '14.300',
   'timeMin': '5.866',
   'iStartUnitCode': None,
   'iEndUnitCode': None,
   'stdTime': '00  ',
   'startUnitNm': '서울',
   'endUnitNm': '수원신갈',
   'startEndStdTypeNm': '도착기준통행시간',
   'tcsCarTypeName': '1종',
   'tcsCarTypeCode': '1',
   'tcsCarTypeDivName': '소형차',
   'tcsCarTypeDivCode': '1',
   'efcvTrfl': '203',
   'iStartEndStdTypeCode': None},
  {'pageNo': None,
   'numOfRows': None,
   'startUnitCode': '101 ',
   'endUnitCode': '103 ',
   'stdDate': '20230807',
   'sumTmUnitTypeCode': None,
   'startEndStdTypeCode': '2',
   'timeAvg': '8.25',
   'timeMax': '14.300',
   'timeMin': '5.866',
   'iStartUnitCode': None,
   'iEndUnitCode': None,
   'stdTime': '00:00',
   'startUnitNm

## 미션
- JSON 데이터를 pandas 데이터프레임으로 가공

In [7]:
import pandas as pd

cars = json_df['realUnitTrtmVO']
pd.DataFrame(cars)

,pageNo,numOfRows,startUnitCode,endUnitCode,stdDate,sumTmUnitTypeCode,startEndStdTypeCode,timeAvg,timeMax,timeMin,...,stdTime,startUnitNm,endUnitNm,startEndStdTypeNm,tcsCarTypeName,tcsCarTypeCode,tcsCarTypeDivName,tcsCarTypeDivCode,efcvTrfl,iStartEndStdTypeCode
0,None,None,101,103,20230807,None,2,7.8,14.300,5.866,...,00,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,203,None
1,None,None,101,103,20230807,None,2,8.25,14.300,5.866,...,00:00,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,62,None
2,None,None,101,103,20230807,None,2,8.316666666666666,11.266,5.916,...,00:05,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,16,None
3,None,None,101,103,20230807,None,2,8.166666666666666,9.433,5.866,...,00:10,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,18,None
4,None,None,101,103,20230807,None,2,9.5,12.700,9.483,...,05:45,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,172,None
5,None,None,101,103,20230807,None,2,10.066666666666666,12.700,10.033,...,05:50,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,35,None
6,None,None,101,103,20230807,None,2,10.3,12.583,9.883,...,05:55,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,47,None
7,None,None,101,103,20230807,None,2,9.5,16.183,10.550,...,06,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,554,None
8,None,None,101,103,20230807,None,2,9.958333333333334,13.416,10.550,...,06:00,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,170,None
9,None,None,101,103,20230807,None,2,10.75,13.416,10.950,...,06:05,서울,수원신갈,도착기준통행시간,1종,1,소형차,1,42,None


In [41]:
%%timeit

import requests
import lxml
from bs4 import BeautifulSoup

url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103&numOfRows=10&pageNo=1"
req = requests.get(url)
soup = BeautifulSoup(req.text, "lxml")
# print(soup.prettify())

stddate_list = soup.find_all("stddate")
timeAvg_list = soup.find_all("timeavg")

stddate = []
avgTime = []
for date, timeAvg in zip(stddate_list, timeAvg_list):
    # print(date.get_text(), timeAvg.get_text())
    stddate.append(date.get_text())
    avgTime.append(timeAvg.get_text())

df = pd.DataFrame({"date":stddate, "avgTime":avgTime})
df

1.3 s ± 189 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit

import pandas as pd
import xml.etree.ElementTree as ET
import requests
url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103"
response = requests.get(url)
root = ET.fromstring(response.content)
data_1 = [
    {element.tag: element.text for element in realUnitTrtmVO}
    for realUnitTrtmVO in root.findall('.//realUnitTrtmVO')
]
pd.DataFrame(data_1)

1.81 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit

import requests
import pandas as pd
import xml.etree.ElementTree as ET
url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103&numOfRows=10&pageNo=1"
req2 = requests.get(url)
root = ET.fromstring(req2.content)
print(root)
data_list = []
for item in root.findall(".//realUnitTrtmVO"):
    data = {}
    for child in item:
        data[child.tag] = child.text
    data_list.append(data)
df_xml = pd.DataFrame(data_list)
df_xml

<Element 'data' at 0x000001E5B9AF7B80>
<Element 'data' at 0x000001E5B9AF7D10>
<Element 'data' at 0x000001E5B9AF7AE0>
<Element 'data' at 0x000001E5B9AF7F90>
<Element 'data' at 0x000001E5B9AF77C0>
<Element 'data' at 0x000001E5B9AF7C70>
<Element 'data' at 0x000001E5B9AF7A40>
<Element 'data' at 0x000001E5B9AF7B30>
5.53 s ± 1.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 부동산 실거래가
- 서울 열린데이터 광장에 부동산 데이터 수집
- 공공데이터 포털 부동산 데이터 수집

In [122]:
import requests
import pandas as pd

url3 = "http://openapi.seoul.go.kr:8088/4244466c7563686835334f51457341/json/tbLnOpendataRtmsV/1/10/"
json_url3 = requests.get(url3)
url3_dic = json_url3.json()
pd.DataFrame(url3_dic['tbLnOpendataRtmsV']['row'])

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11710,송파구,10800,문정동,1,대지,0652,0004,힐스테이트에코문정,...,20500,20.79,32.68,4.0,,,2018,오피스텔,중개거래,서울 송파구
1,2023,11560,영등포구,10900,영등포동8가,1,대지,0051,0001,메이준,...,8000,17.44,25.56,6.0,,,2012,오피스텔,중개거래,서울 영등포구
2,2023,11380,은평구,10500,구산동,1,대지,0345,0020,솔빛엘르빌,...,22000,49.30,27.73,2.0,,,2012,연립다세대,중개거래,서울 은평구
3,2023,11620,관악구,10100,봉천동,1,대지,1708,0000,두산,...,80000,59.92,0.00,10.0,,,2000,아파트,중개거래,서울 관악구
4,2023,11500,강서구,10300,화곡동,1,대지,1071,0007,와이앤비,...,22500,25.38,15.19,3.0,,,2017,연립다세대,중개거래,서울 강서구
5,2023,11500,강서구,10300,화곡동,1,대지,0338,0006,가이아팰리스,...,26750,29.99,18.72,2.0,,,2019,연립다세대,중개거래,서울 양천구
6,2023,11440,마포구,12500,성산동,1,대지,0593,0002,상암 수린나 오피스텔,...,16500,17.55,27.64,10.0,,,2018,오피스텔,직거래,
7,2023,11740,강동구,10300,상일동,1,대지,0514,0000,고덕롯데캐슬베네루체,...,105000,59.98,0.00,11.0,,,2019,아파트,중개거래,서울 강동구
8,2023,11620,관악구,10100,봉천동,1,대지,1574,0001,생모리츠타운,...,11400,22.24,30.66,9.0,,,2007,오피스텔,직거래,
9,2023,11620,관악구,10100,봉천동,1,대지,0715,0185,(715-185),...,15000,41.06,31.30,1.0,,,1988,연립다세대,직거래,


In [71]:
import requests
import pandas as pd
import lxml
from bs4 import BeautifulSoup

url4 = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
params ={'serviceKey' : 'YjwuOSErCt4PB%2BANN4eK26d76AIC6dbwF52v%2FNkCmmZdIi4ZSXRSITMWtc2y%2B%2F8gjL6p4%2FBfiNZgoraqpsihDg%3D%3D',
         'LAWD_CD' : '11110', 'DEAL_YMD' : '202212' }

xml_url4 = requests.get(url4, params=params)
xml_url4.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'

In [123]:
import requests
import pandas as pd

serviceKey = 'YjwuOSErCt4PB%2BANN4eK26d76AIC6dbwF52v%2FNkCmmZdIi4ZSXRSITMWtc2y%2B%2F8gjL6p4%2FBfiNZgoraqpsihDg%3D%3D'
pageNo = 1
nomOfRows = 1000
LAWD_CD = 11560
DEAL_YMD = 202307
url = f'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey={serviceKey}&pageNo={pageNo}&numOfRows=10&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}'
req = requests.get(url)
req

<Response [200]>

In [63]:
!pip install xmltodict

In [124]:
import pandas as pd
import xmltodict

data_dict = xmltodict.parse(req.text)
pd.DataFrame(data_dict['response']['body']['items']['item'])

,거래금액,거래유형,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명코드,...,일,일련번호,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부,도로명지상지하코드
0,"41,700",중개거래,2020,2023,경인로,00900,00000,11560,00,3000028,...,1,11560-2153,30.2696,서울 영등포구,94-2,11560,3,None,None,NaN
1,"30,500",직거래,2022,2023,경인로114길,00006,00000,11560,00,4154044,...,31,11560-2281,31.19,None,92-3,11560,5,None,None,0
2,"62,260",중개거래,2022,2023,버드나루로,00005,00000,11560,00,3118010,...,17,11560-2270,38.441,서울 영등포구,139,11560,8,None,None,0
3,"62,260",중개거래,2022,2023,버드나루로,00005,00000,11560,00,3118010,...,17,11560-2270,38.441,서울 영등포구,139,11560,8,23.08.01,O,0
4,"10,400",중개거래,2014,2023,영중로6길,00012,00000,11560,01,4154816,...,7,11560-1680,13.53,서울 영등포구,19-2,11560,7,None,None,0
5,"145,700",중개거래,2017,2023,국회대로54길,00010,00000,11560,00,4154084,...,4,11560-1955,84.75,서울 영등포구,203,11560,12,None,None,NaN
6,"145,000",중개거래,2017,2023,국회대로54길,00010,00000,11560,00,4154084,...,6,11560-1955,84.75,서울 영등포구,203,11560,11,None,None,NaN
7,"120,000",중개거래,2017,2023,국회대로54길,00010,00000,11560,00,4154084,...,6,11560-1955,59.91,"서울 송파구, 서울 영등포구",203,11560,13,None,None,NaN
8,"81,000",중개거래,1999,2023,영중로,00145,00000,11560,06,3118024,...,1,11560-7,59.91,서울 영등포구,76-1,11560,6,None,None,0
9,"102,000",중개거래,1999,2023,영중로,00145,00000,11560,06,3118024,...,4,11560-7,84.87,"서울 양천구, 서울 영등포구",76-1,11560,6,None,None,0


In [90]:
import requests
import json
import pandas as pd

SERVICE_KEY = '4244466c7563686835334f51457341'
url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/1/1000/2023/11560'
req = requests.get(url)
req

<Response [200]>

In [91]:
content = req.json()
result = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])
# result
content

{'tbLnOpendataRtmsV': {'list_total_count': 1862,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'ACC_YEAR': '2023',
    'SGG_CD': '11560',
    'SGG_NM': '영등포구',
    'BJDONG_CD': '10900',
    'BJDONG_NM': '영등포동8가',
    'LAND_GBN': '1',
    'LAND_GBN_NM': '대지',
    'BONBEON': '0051',
    'BUBEON': '0001',
    'BLDG_NM': '메이준',
    'DEAL_YMD': '20230804',
    'OBJ_AMT': '8000',
    'BLDG_AREA': 17.44,
    'TOT_AREA': 25.56,
    'FLOOR': 6.0,
    'RIGHT_GBN': '',
    'CNTL_YMD': '',
    'BUILD_YEAR': '2012',
    'HOUSE_TYPE': '오피스텔',
    'REQ_GBN': '중개거래',
    'RDEALER_LAWDNM': '서울 영등포구'},
   {'ACC_YEAR': '2023',
    'SGG_CD': '11560',
    'SGG_NM': '영등포구',
    'BJDONG_CD': '13200',
    'BJDONG_NM': '신길동',
    'LAND_GBN': '1',
    'LAND_GBN_NM': '대지',
    'BONBEON': '4934',
    'BUBEON': '0000',
    'BLDG_NM': '삼성래미안',
    'DEAL_YMD': '20230803',
    'OBJ_AMT': '75000',
    'BLDG_AREA': 59.33,
    'TOT_AREA': 0.0,
    'FLOOR': 5.0,
    'RIGHT_GBN': '',
    'CNTL_YMD'

In [119]:
import requests
import json
import pandas as pd

SERVICE_KEY = '4244466c7563686835334f51457341'
s_index = 1
df_list = []

for i in range(12):
    s_index += 1000 * i
    url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{s_index}/{s_index + 999}/'
    req = requests.get(url)
    content = req.json()
    if 'tbLnOpendataRtmsV' in content.keys():
        result = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])
        df_list.append(result)
    else:
        break

pd.concat(df_list)

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11710,송파구,10800,문정동,1,대지,0652,0004,힐스테이트에코문정,...,20500,20.79,32.68,4.0,,,2018,오피스텔,중개거래,서울 송파구
1,2023,11560,영등포구,10900,영등포동8가,1,대지,0051,0001,메이준,...,8000,17.44,25.56,6.0,,,2012,오피스텔,중개거래,서울 영등포구
2,2023,11380,은평구,10500,구산동,1,대지,0345,0020,솔빛엘르빌,...,22000,49.30,27.73,2.0,,,2012,연립다세대,중개거래,서울 은평구
3,2023,11620,관악구,10100,봉천동,1,대지,1708,0000,두산,...,80000,59.92,0.00,10.0,,,2000,아파트,중개거래,서울 관악구
4,2023,11500,강서구,10300,화곡동,1,대지,1071,0007,와이앤비,...,22500,25.38,15.19,3.0,,,2017,연립다세대,중개거래,서울 강서구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022,11680,강남구,10700,신사동,1,대지,0588,0016,현대썬앤빌,...,36030,19.77,28.59,14.0,,,2015,오피스텔,직거래,
996,2022,11620,관악구,10100,봉천동,1,대지,0296,0000,낙성대현대(296),...,75000,84.97,0.00,3.0,,,1998,아파트,중개거래,서울 관악구
997,2022,11230,동대문구,10900,휘경동,1,대지,0322,0002,회기역 더이스트팰리스,...,22100,16.69,28.65,6.0,,,2020,오피스텔,중개거래,서울 동대문구
998,2022,11680,강남구,10700,신사동,1,대지,0588,0016,현대썬앤빌,...,34030,19.77,28.59,14.0,,,2015,오피스텔,직거래,


In [121]:
import requests
import json
import pandas as pd

SERVICE_KEY = '4244466c7563686835334f51457341'
s_index = 1
df_list = []

for i in range(12):
    s_index += 1000 * i
    url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{s_index}/{s_index + 999}/'
    req = requests.get(url)
    content = req.json()
    result = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])
    df_list.append(result)

pd.concat(df_list).reset_index(drop = True)

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11710,송파구,10800,문정동,1,대지,0652,0004,힐스테이트에코문정,...,20500,20.79,32.68,4.0,,,2018,오피스텔,중개거래,서울 송파구
1,2023,11560,영등포구,10900,영등포동8가,1,대지,0051,0001,메이준,...,8000,17.44,25.56,6.0,,,2012,오피스텔,중개거래,서울 영등포구
2,2023,11380,은평구,10500,구산동,1,대지,0345,0020,솔빛엘르빌,...,22000,49.30,27.73,2.0,,,2012,연립다세대,중개거래,서울 은평구
3,2023,11620,관악구,10100,봉천동,1,대지,1708,0000,두산,...,80000,59.92,0.00,10.0,,,2000,아파트,중개거래,서울 관악구
4,2023,11500,강서구,10300,화곡동,1,대지,1071,0007,와이앤비,...,22500,25.38,15.19,3.0,,,2017,연립다세대,중개거래,서울 강서구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2022,11680,강남구,10700,신사동,1,대지,0588,0016,현대썬앤빌,...,36030,19.77,28.59,14.0,,,2015,오피스텔,직거래,
11996,2022,11620,관악구,10100,봉천동,1,대지,0296,0000,낙성대현대(296),...,75000,84.97,0.00,3.0,,,1998,아파트,중개거래,서울 관악구
11997,2022,11230,동대문구,10900,휘경동,1,대지,0322,0002,회기역 더이스트팰리스,...,22100,16.69,28.65,6.0,,,2020,오피스텔,중개거래,서울 동대문구
11998,2022,11680,강남구,10700,신사동,1,대지,0588,0016,현대썬앤빌,...,34030,19.77,28.59,14.0,,,2015,오피스텔,직거래,


In [118]:
import requests
import json
import pandas as pd

SERVICE_KEY = '4244466c7563686835334f51457341'
data = None
for j in range(1,13):
    url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{1+((j-1)*1000)}/{j*1000}'
    # print(url)
    req = requests.get(url)
    content = req.json()
    con = content['tbLnOpendataRtmsV']['row']
    result = pd.DataFrame(con)
    data = pd.concat([data, result])
data = data.reset_index(drop=True)
data

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11710,송파구,10800,문정동,1,대지,0652,0004,힐스테이트에코문정,...,20500,20.79,32.68,4.0,,,2018,오피스텔,중개거래,서울 송파구
1,2023,11560,영등포구,10900,영등포동8가,1,대지,0051,0001,메이준,...,8000,17.44,25.56,6.0,,,2012,오피스텔,중개거래,서울 영등포구
2,2023,11380,은평구,10500,구산동,1,대지,0345,0020,솔빛엘르빌,...,22000,49.30,27.73,2.0,,,2012,연립다세대,중개거래,서울 은평구
3,2023,11620,관악구,10100,봉천동,1,대지,1708,0000,두산,...,80000,59.92,0.00,10.0,,,2000,아파트,중개거래,서울 관악구
4,2023,11500,강서구,10300,화곡동,1,대지,1071,0007,와이앤비,...,22500,25.38,15.19,3.0,,,2017,연립다세대,중개거래,서울 강서구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,2023,11740,강동구,10900,천호동,1,대지,0447,0017,강동역신동아파밀리에,...,97000,94.61,0.00,8.0,,,2015,아파트,중개거래,서울 강동구
11996,2023,11215,광진구,10300,구의동,1,대지,0249,0024,현대빌라(249-24),...,37000,61.71,28.65,1.0,,,1995,연립다세대,중개거래,서울 광진구
11997,2023,11380,은평구,10700,응암동,1,대지,0081,0210,로얄쉐르빌2동,...,21500,46.09,21.50,1.0,,,2002,연립다세대,중개거래,서울 마포구
11998,2023,11470,양천구,10200,목동,1,대지,0497,0024,경우하이츠맨션,...,19300,61.51,29.70,-1.0,,,2000,연립다세대,중개거래,서울 양천구
